In [1]:
__author__ = 'Alexander Soroka, soroka.a.m@gmail.com'
__copyright__ = """Copyright 2020 Alexander Soroka"""


import argparse
import glob
import numpy as np
import tensorflow as tf
import time
from tensorflow.python import keras as keras
from tensorflow.python.keras.callbacks import LearningRateScheduler


LOG_DIR = 'logs'
SHUFFLE_BUFFER = 4
BATCH_SIZE = 64
NUM_CLASSES = 6
PARALLEL_CALLS=4
RESIZE_TO = 224
TRAINSET_SIZE = 14034
VALSET_SIZE = 3000


def parse_proto_example(proto):
    keys_to_features = {
        'image/encoded': tf.io.FixedLenFeature((), tf.string, default_value=''),
        'image/class/label': tf.io.FixedLenFeature([], tf.int64, default_value=tf.zeros([], dtype=tf.int64))
    }
    example = tf.io.parse_single_example(proto, keys_to_features)
    example['image'] = tf.image.decode_jpeg(example['image/encoded'], channels=3)
    example['image'] = tf.image.convert_image_dtype(example['image'], dtype=tf.float32)
    example['image'] = tf.image.resize(example['image'], tf.constant([RESIZE_TO, RESIZE_TO]))
    return example['image'], tf.one_hot(example['image/class/label'], depth=NUM_CLASSES)


def normalize(image, label):
    return tf.image.per_image_standardization(image), label

def resize(image, label):
    return tf.image.resize(image, tf.constant([RESIZE_TO, RESIZE_TO])), label

def create_dataset(filenames, batch_size):
    """Create dataset from tfrecords file
    :tfrecords_files: Mask to collect tfrecords file of dataset
    :returns: tf.data.Dataset
    """
    return tf.data.TFRecordDataset(filenames)\
        .map(parse_proto_example)\
        .map(resize)\
        .map(normalize)\
        .batch(batch_size)\
        .prefetch(batch_size)


def build_model():

    new_model = keras.models.load_model('/content/drive/My Drive/SMOMI/modellab3_3-2.h5')
    
    new_model.trainable = True

    return new_model
    
    
      


def main():
    #args = argparse.ArgumentParser()
    #args.add_argument('--train', type=str, help='Glob pattern to collect train tfrecord files')
    #args.add_argument('--test', type=str, help='Glob pattern to collect test tfrecord files')
    #args = args.parse_args()
    train_path = '/content/drive/My Drive/SMOMI/dataset/train*'
    test_path = '/content/drive/My Drive/SMOMI/dataset/val*'

    train_dataset = create_dataset(glob.glob(train_path), BATCH_SIZE)
    validation_dataset = create_dataset(glob.glob(test_path), BATCH_SIZE)

    model = build_model()

    model.compile(
        optimizer=tf.optimizers.SGD(lr=0.000000000004, momentum=0.9),
        loss=tf.keras.losses.categorical_crossentropy,
        metrics=[tf.keras.metrics.categorical_accuracy],
    )

    log_dir='/content/drive/My Drive/SMOMI/{}/lab3_3-2/ilcd-{}'.format(LOG_DIR, time.time())
    model.fit(
        train_dataset,
        epochs=150,
        validation_data=validation_dataset,
        callbacks=[
            tf.keras.callbacks.TensorBoard(log_dir),
        ]
    )
    model.save('/content/drive/My Drive/SMOMI/modellab3_3-2-full.h5')


if __name__ == '__main__':
    main()

Epoch 1/150
      1/Unknown - 0s 646us/step - loss: 3.1222 - categorical_accuracy: 0.1094WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
      2/Unknown - 1s 373ms/step - loss: 3.1540 - categorical_accuracy: 0.1250WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0911s vs `on_train_batch_end` time: 0.6528s). Check your callbacks.
220/220 [==============================] - 82s 372ms/step - loss: 2.1569 - categorical_accuracy: 0.1828 - val_loss: 0.4946 - val_categorical_accuracy: 0.6630
Epoch 2/150
220/220 [==============================] - 70s 319ms/step - loss: 2.1569 - categorical_accuracy: 0.1828 - val_loss: 0.5275 - val_categorical_accuracy: 0.6513
Epoch 3/150
220/220 [==============================] 

KeyboardInterrupt: ignored